# Assignment 3
(Assignment groups 2 and 10)

- Jasper Bosschart s2562685
- Marina Stefanova s2610604
- Nagham Mekky s3096564
- Sven Sonneveld s2594854
- Hynek Noll s3102181

In [2]:
import matplotlib.pyplot as plt
import numpy as np
import csv
import pandas as pd

# Motivation and Goals

# Methodology and Results 
Our goal is to analyze leaked chat logs of a ransomware group Conti.

1) How many total records are in the data?  

In [5]:
file = open('jabber_chat_2021_2022_translated.csv',encoding='utf8')
reader = csv.reader(file)
record_count= len(list(reader))
print("Number of records: ", record_count)

Number of records:  60774


3) How many messages are sent originally in Russian, and how many are sent in English?

In [4]:
russian_count = 0
english_count = 0
# Read the data from the csv file
with open('jabber_chat_2021_2022_translated.csv','r',encoding='utf8') as csvfile:
    lines = csv.reader(csvfile, delimiter=',')
    for row in lines:
        if (row[6]=='ru'):
            russian_count += 1
        elif (row[6]=='en'):
            english_count += 1
print("Number of Russian messages: ", russian_count)
print("Number of English messages: ", english_count)

Number of Russian messages:  32706
Number of English messages:  4314


# Conclusion and Reflections